In [1]:
import pandas as pd
import glob

In [2]:
plink2 = '/home/vitaled2/.genotools/misc/executables/plink2'
project_dir = '/home/vitaled2/gp2_carriers'
data_dir = f'{project_dir}/data'
report_path = f'{data_dir}/NBA_Report.csv'
geno_dir = f'{data_dir}/raw_genotypes'
labels = ['AAC', 'AFR', 'AJ', 'AMR', 'CAH', 'CAS', 'EAS', 'EUR', 'FIN','MDE', 'SAS']
key = pd.read_csv(f'data/nba_app_key.csv')

In [4]:
# get snps list
report = pd.read_csv(report_path)
report.columns = [x.strip() for x in report.columns]
report.drop(columns=['GP2ID','Cohort Code', 'Local ID',	'PI','PD EUR PRS (GWAS vXX)','NBA derived Ancestry'], inplace=True)

snp_df = report.loc[~report['Unnamed: 0'].isna()]
snp_df.set_index('Unnamed: 0', inplace=True)
snp_df = snp_df.transpose()
snp_df.reset_index(inplace=True)
snp_df.rename(columns={'index':'snp'}, inplace=True)
snp_df.drop(columns=['Alternative variant name'], inplace=True)
snp_df.columns.name = None
snp_df.columns = ['snp','rsid','hg19','hg38','ancestry']
snp_df = snp_df.loc[~snp_df.hg38.isna()]

pvar_total = pd.DataFrame()
for label in labels:
    label_geno_path = f'{geno_dir}/{label}/{label}_release9_vwb'
    pvar = pd.read_csv(f'{label_geno_path}.pvar', sep='\t', dtype={'#CHROM':str})
    pvar.columns = ['chrom', 'pos', 'id', 'a1', 'a2']
    pvar.loc[:,'snpid1'] = pvar['chrom'] + ':' + pvar['pos'].astype(str) + ':' + pvar['a1'] + ':' + pvar['a2']
    pvar.loc[:,'snpid2'] = pvar['chrom'] + ':' + pvar['pos'].astype(str) + ':' + pvar['a2'] + ':' + pvar['a1']
    pvar_sub1 = pvar.loc[pvar['snpid1'].isin(snp_df['hg38'])].copy()
    pvar_sub1.loc[:,'snpid'] = pvar_sub1['snpid1']
    pvar_sub2 = pvar.loc[pvar['snpid2'].isin(snp_df['hg38'])].copy()
    pvar_sub2.loc[:,'snpid'] = pvar_sub2['snpid2']
    pvar_total = pd.concat([pvar_total, pvar_sub1, pvar_sub2], ignore_index=True)
pvar_total.drop(columns=['snpid1','snpid2'], inplace=True)

# pvar_total[['id']].drop_duplicates().to_csv(f'{data_dir}/snps.txt', header=False, index=False)

snp_df_out = pvar_total.merge(snp_df, how='left', left_on='snpid', right_on='hg38')
snp_df_out.drop(columns=['chrom', 'pos', 'a1', 'a2'], inplace=True)
snp_df_out.rename(columns={'snp':'variant'}, inplace=True)

snp_df_out['snp_name'] = snp_df_out['variant'].str.extract(f'\((.*?)\)')

snp_df_out.loc[:,'locus'] = snp_df_out.variant.str.split(' ', expand=True)[:][0]
snp_df_out.loc[:,'snp_name_full'] = snp_df_out['locus'] + '_' + snp_df_out['snp_name'] + '_' + snp_df_out['id']
snp_df_out.drop(columns=['snpid'], inplace=True)
snp_df_out.drop_duplicates(inplace=True)
snp_df_out.to_csv(f'{data_dir}/snps_out.csv', index=False)





In [3]:
from carriers_api.app.carriers import extract_carriers, combine_carrier_files

output_dir = f'{data_dir}/api_test/outputs'

results_by_label = {}
for label in labels:
    results = extract_carriers(
        geno_path=f'{geno_dir}/{label}/{label}_release9_vwb',
        snplist_path=f'{data_dir}/snps_out.csv',
        out_path=f'{output_dir}/{label}',
        return_dfs=True
    )
    results_by_label[label] = results

combined_results = combine_carrier_files(
    results_by_label=results_by_label,
    key_file=f'{data_dir}/nba_app_key.csv',
    output_dir=output_dir
)

PLINK v2.0.0-a.6.3LM AVX2 Intel (3 Dec 2024)       cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/vitaled2/gp2_carriers/data/raw_genotypes/AAC/AAC_release9_vwb_snps.log.
Options in effect:
  --export Av
  --extract /home/vitaled2/gp2_carriers/data/raw_genotypes/AAC/AAC_release9_vwb_temp_snps.txt
  --freq
  --out /home/vitaled2/gp2_carriers/data/raw_genotypes/AAC/AAC_release9_vwb_snps
  --pfile /home/vitaled2/gp2_carriers/data/raw_genotypes/AAC/AAC_release9_vwb

Start time: Fri Feb 14 18:48:04 2025
32105 MiB RAM detected, ~30169 available; reserving 16052 MiB for main
workspace.
Using up to 8 compute threads.
1230 samples (724 females, 506 males; 1230 founders) loaded from
/home/vitaled2/gp2_carriers/data/raw_genotypes/AAC/AAC_release9_vwb.psam.
1896835 variants loaded from
/home/vitaled2/gp2_carriers/data/raw_genotypes/AAC/AAC_release9_vwb.pvar.
1 binary phenotype loaded (341 cases, 837 controls).
--extract: 2

In [4]:
carriers_string = pd.read_csv(combined_results['carriers_string'])
var_info = pd.read_csv(combined_results['var_info'])
var_info

,id,rsid,hg19,hg38,ancestry,CHR,SNP,(C)M,POS,COUNTED,...,ALT_FREQS_EAS,OBS_CT_EAS,ALT_FREQS_EUR,OBS_CT_EUR,ALT_FREQS_FIN,OBS_CT_FIN,ALT_FREQS_MDE,OBS_CT_MDE,ALT_FREQS_SAS,OBS_CT_SAS
0,1:155204987-C-T,rs80356771,1:155204987:G:A,1:155235196:G:A,NaN,1,1:155204987-C-T,0,155235196,G,...,0.001949,11800,0.000664,102480,0.000000,274,0.000000,1732,0.000000,1462
1,Seq_rs80356771.1_ilmnrev_ilmnF2BT,rs80356771,1:155204987:G:A,1:155235196:G:A,NaN,1,Seq_rs80356771.1_ilmnrev_ilmnF2BT,0,155235196,G,...,0.001949,11798,0.000684,102394,0.000000,274,0.000000,1732,0.000000,1460
2,rs80356771,rs80356771,1:155204987:G:A,1:155235196:G:A,NaN,1,rs80356771,0,155235196,G,...,0.001949,11798,0.000683,102492,0.000000,274,0.000000,1732,0.000000,1462
3,exm106220,rs75548401,1:155206037:G:A,1:155236246:G:A,AMISH/ EUR,1,exm106220,0,155236246,G,...,0.000339,11798,0.012223,102434,0.029412,272,0.000577,1732,0.000684,1462
4,rs75548401,rs75548401,1:155206037:G:A,1:155236246:G:A,AMISH/ EUR,1,rs75548401,0,155236246,G,...,0.000254,11788,0.012061,102226,0.033088,272,0.000579,1728,0.000685,1460
5,seq_rs75548401,rs75548401,1:155206037:G:A,1:155236246:G:A,AMISH/ EUR,1,seq_rs75548401,0,155236246,G,...,0.000254,11804,0.012260,102446,0.029412,272,0.000577,1732,0.000684,1462
6,Seq_rs34424986.1_ilmnrev_ilmnF2BT,rs34424986,NaN,6:161785820:G:A,MULT,6,Seq_rs34424986.1_ilmnrev_ilmnF2BT,0,161785820,G,...,0.000000,11796,0.001060,101902,0.000000,274,0.000000,1732,0.000000,1460
7,exm593505,rs34424986,NaN,6:161785820:G:A,MULT,6,exm593505,0,161785820,G,...,0.000170,11794,0.003641,102452,0.000000,274,0.000000,1732,0.000684,1462
8,rs34424986,rs34424986,NaN,6:161785820:G:A,MULT,6,rs34424986,0,161785820,G,...,0.000085,11798,0.003815,102480,0.000000,274,0.000000,1732,0.000684,1462
9,12:40692927-A-G,rs34805604,NaN,12:40299125:A:G,MULT,12,12:40692927-A-G,0,40299125,A,...,0.000000,11802,0.000000,102486,0.000000,274,0.000000,1732,0.000000,1462


In [9]:
carriers_string[carriers_string['GBA1_p.R502C_1:155204987-C-T']!='WT/WT']

,IID,study,ancestry,GBA1_p.R502C_1:155204987-C-T,GBA1_p.R502C_Seq_rs80356771.1_ilmnrev_ilmnF2BT,GBA1_p.R502C_rs80356771,GBA1_p.T408M_exm106220,GBA1_p.T408M_rs75548401,GBA1_p.T408M_seq_rs75548401,PRKN_R275W_Seq_rs34424986.1_ilmnrev_ilmnF2BT,...,LRRK2_R1441H_Seq_rs34995376,LRRK2_Y1699C_12:40714916-A-G,LRRK2_Y1699C_Seq_rs35801418,LRRK2_G2019S_exm994671,LRRK2_G2019S_rs34637584,LRRK2_G2019S_seq_rs34637584,LRRK2_G2385D_exm994721,LRRK2_G2385D_rs34778348,LRRK2_G2385D_seq_rs34778348,SNCA_A30P_rs104893878
32,BLAACPD-KPM_000032,BLAACPD-KPM,AAC,WT/p.R502C,WT/p.R502C,WT/p.R502C,WT/WT,WT/WT,WT/WT,WT/WT,...,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT
1412,BLAACPD-UFL_000002,BLAACPD-UFL,AFR,WT/p.R502C,WT/p.R502C,WT/p.R502C,WT/WT,WT/WT,WT/WT,WT/WT,...,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT
5121,LCC_000008,LCC,AJ,./.,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,...,WT/WT,WT/WT,WT/WT,WT/G2019S,WT/G2019S,WT/G2019S,WT/WT,WT/WT,WT/WT,WT/WT
9269,LARGEPD_001828,LARGEPD,AMR,./.,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,...,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT
10689,PDGNRTN_007257,PDGNRTN,AMR,WT/p.R502C,WT/p.R502C,WT/p.R502C,WT/WT,WT/WT,WT/WT,WT/WT,...,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69623,VIPD_000214,VIPD,EUR,WT/p.R502C,WT/p.R502C,WT/p.R502C,WT/WT,WT/WT,WT/WT,WT/WT,...,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT
69760,VIPD_000369,VIPD,EUR,WT/p.R502C,WT/p.R502C,WT/p.R502C,WT/WT,WT/WT,WT/WT,WT/WT,...,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT
69881,VIPD_000512,VIPD,EUR,WT/p.R502C,WT/p.R502C,WT/p.R502C,WT/WT,WT/WT,WT/WT,WT/WT,...,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT
70045,VIPD_000741,VIPD,EUR,WT/p.R502C,WT/p.R502C,WT/p.R502C,WT/WT,WT/WT,WT/WT,WT/WT,...,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT,WT/WT


In [11]:
def verify_genotype(raw_value, string_value, snp_name):
    if pd.isna(raw_value):
        return string_value == './.'
    raw_value = int(raw_value)
    if raw_value == 2:
        return string_value == "WT/WT"
    elif raw_value == 1:
        return string_value.startswith("WT/")
    elif raw_value == 0:
        return string_value.count(snp_name.split('_')[1]) == 2
    else:
        return string_value == ":/:"

def validate_carrier_data(traw_dir, combined_file, snp_info_file, samples_to_check=None):
    """
    Validate that combined carrier data matches original traw files.
    
    Args:
        traw_dir: Directory containing traw files
        combined_file: Path to combined carriers file
        snp_info_file: Path to SNP info file
        samples_to_check: List of sample IDs to check (None for all)
    """
    carriers = pd.read_csv(combined_file)
    snp_df = pd.read_csv(snp_info_file)
    
    if samples_to_check is None:
        samples_to_check = carriers['IID'].unique()[:5]  # Check first 5 samples by default
    
    mismatches = []
    for sample_id in samples_to_check:
        # Get ancestry for this sample
        ancestry = carriers.loc[carriers['IID'] == sample_id, 'ancestry'].iloc[0]
        
        # Read corresponding traw file
        traw_path = f'{traw_dir}/{ancestry}/{ancestry}_release9_vwb_snps.traw'
        traw = pd.read_csv(traw_path, sep='\t')
        traw_merged = snp_df.merge(traw, how='left', left_on='id', right_on='SNP')
        
        # Check each SNP
        for snp in snp_df['snp_name_full']:
            raw_value = traw_merged.loc[
                traw_merged['snp_name_full'] == snp, 
                f'0_{sample_id}'
            ].iloc[0]
            
            combined_value = carriers.loc[
                carriers['IID'] == sample_id,
                snp
            ].iloc[0]
            
            if not verify_genotype(raw_value, combined_value, snp):
                mismatches.append({
                    'sample': sample_id,
                    'ancestry': ancestry,
                    'snp': snp,
                    'raw': raw_value,
                    'combined': combined_value
                })
    
    if mismatches:
        print("Found mismatches:")
        print(pd.DataFrame(mismatches))
    else:
        print("All checked genotypes match!")

# Use it like this:
validate_carrier_data(
    traw_dir=geno_dir,
    combined_file=f'{output_dir}/carriers_string_full.csv',
    snp_info_file=f'{data_dir}/snps_out.csv',
    samples_to_check=['BLAACPD-KPM_000032']
)

All checked genotypes match!


In [13]:

# from carriers_api.app.carriers import extract_carriers

# output_dir = f'{data_dir}/api_test/outputs'
# carriers_string_full_path = f'{output_dir}/carriers_string_full.csv'
# carriers_int_full_path = f'{output_dir}/carriers_int_full.csv'
# var_info_full_path = f'{output_dir}/var_info_full.csv'

# carriers_string_full = pd.DataFrame()
# carriers_int_full = pd.DataFrame()
# label_var_info_full = pd.DataFrame()

# for label in labels:
#     results = extract_carriers(
#         geno_path=f'{geno_dir}/{label}/{label}_release9_vwb',
#         snplist_path=f'{data_dir}/snps_out.csv',
#         out_path=f'{output_dir}/{label}'
#     )

#     label_carriers_string = results['carriers_string']
#     label_carriers_int = results['carriers_int']
#     label_var_info = results['var_info']

#     label_var_info_full = pd.concat([label_var_info_full, label_var_info], ignore_index=True)

#     carriers_string = pd.read_csv(label_carriers_string)
#     carriers_string['IID'] = carriers_string['IID'].str.replace('0_', '')
#     carriers_string.loc[:,'ancestry'] = label
#     carriers_string_full = pd.concat([carriers_string_full, carriers_string], ignore_index=True)
    
#     carriers_int = pd.read_csv(label_carriers_int)
#     carriers_int['IID'] = carriers_int['IID'].str.replace('0_', '')
#     carriers_int.loc[:,'ancestry'] = label
#     carriers_int_full = pd.concat([carriers_int_full, carriers_int], ignore_index=True)

# variant_columns = [x for x in carriers_string_full.columns if x not in ['IID','ancestry']]
# carriers_string_full_out = carriers_string_full[['IID', 'ancestry'] + variant_columns]
# carriers_string_full_out[variant_columns] = carriers_string_full_out[variant_columns].fillna('./.')
# carriers_string_full_out_merge = carriers_string_full_out.merge(key[['IID','study']], how='left', on='IID')
# carriers_string_final = carriers_string_full_out_merge[['IID', 'study', 'ancestry'] + variant_columns]
# carriers_string_final.to_csv(carriers_string_full_path, index=False)

# # Process and save final integer format
# carriers_int_full_out = carriers_int_full[['IID', 'ancestry'] + [x for x in carriers_int_full.columns if x not in ['IID','ancestry']]]
# carriers_int_full_out_merge = carriers_int_full_out.merge(key[['IID','study']], how='left', on='IID')
# carriers_int_final = carriers_int_full_out_merge[['IID', 'study', 'ancestry'] + variant_columns]
# carriers_int_final.to_csv(carriers_int_full_path, index=False)




In [17]:
# carriers_string_full_path = f'{data_dir}/api_test/outputs/carriers_string_full.csv'
# carriers_int_full_path = f'{data_dir}/api_test/outputs/carriers_int_full.csv'

# carriers_string_full = pd.DataFrame()
# carriers_int_full = pd.DataFrame()

# for label in labels:
#     label_carriers_string = f'{data_dir}/api_test/outputs/{label}_carriers_string.csv'
#     label_carriers_int = f'{data_dir}/api_test/outputs/{label}_carriers_int.csv'
    
#     carriers_string = pd.read_csv(label_carriers_string)
#     carriers_string['IID'] = carriers_string['IID'].str.replace('0_', '')
#     carriers_string.loc[:,'ancestry'] = label
#     carriers_string_full = pd.concat([carriers_string_full, carriers_string], ignore_index=True)
    
#     carriers_int = pd.read_csv(label_carriers_int)
#     carriers_int['IID'] = carriers_int['IID'].str.replace('0_', '')
#     carriers_int.loc[:,'ancestry'] = label
#     carriers_int_full = pd.concat([carriers_int_full, carriers_int], ignore_index=True)

# # Process and save final string format
# variant_columns = [x for x in carriers_string_full.columns if x not in ['IID','ancestry']]
# carriers_string_full_out = carriers_string_full[['IID', 'ancestry'] + variant_columns]
# carriers_string_full_out[variant_columns] = carriers_string_full_out[variant_columns].fillna('./.')
# carriers_string_full_out_merge = carriers_string_full_out.merge(key[['IID','study']], how='left', on='IID')
# carriers_string_final = carriers_string_full_out_merge[['IID', 'study', 'ancestry'] + variant_columns]
# carriers_string_final.to_csv(carriers_string_full_path, index=False)

# # Process and save final integer format
# carriers_int_full_out = carriers_int_full[['IID', 'ancestry'] + [x for x in carriers_int_full.columns if x not in ['IID','ancestry']]]
# carriers_int_full_out_merge = carriers_int_full_out.merge(key[['IID','study']], how='left', on='IID')
# carriers_int_final = carriers_int_full_out_merge[['IID', 'study', 'ancestry'] + variant_columns]
# carriers_int_final.to_csv(carriers_int_full_path, index=False)


In [14]:
# replaced by modules in carriers.py

# for label in labels:
#     label_geno_path = f'{geno_dir}/{label}/{label}_release9_vwb'
#     label_snps_traw = f'{label_geno_path}_snps.traw'
#     label_snps_afreq = f'{label_geno_path}_snps.afreq'
#     label_carriers_var_info = f'{data_dir}/outputs/{label}_carriers_var_info.csv'
#     label_carriers_string = f'{data_dir}/outputs/{label}_carriers_string.csv'
#     label_carriers_int = f'{data_dir}/outputs/{label}_carriers_int.csv'

#     snp_df = pd.read_csv(f'{data_dir}/snps_out.csv')
#     snp_df['id'].to_csv(f'{data_dir}/{label}_snps.txt', header=False, index=False)

#     extract_cmd = f"plink2 --pfile {label_geno_path} --extract {data_dir}/{label}_snps.txt --export Av --freq --out {label_geno_path}_snps"
#     !{extract_cmd}
    
#     traw = pd.read_csv(label_snps_traw, sep='\t')
#     traw_merged = snp_df.merge(traw, how='left', left_on='id', right_on='SNP')
    
#     colnames = [
#         'id',
#         'rsid',
#         'hg19',
#         'hg38',
#         'ancestry', 
#         'CHR',
#         'SNP', 
#         '(C)M', 
#         'POS',
#         'COUNTED',
#         'ALT',
#         'variant',
#         'snp_name', 
#         'locus', 
#         'snp_name_full'
#         ]
#     var_cols = [x for x in colnames if x not in ['snp_name_full']]
#     sample_cols = list(traw_merged.drop(columns=colnames).columns)

#     traw_final = traw_merged.loc[:, colnames + sample_cols]
    
#     traw_out = traw_final.copy()
#     traw_out[sample_cols] = traw_out.apply(
#         lambda row: [genotype_to_string(row[col], row['snp_name']) for col in sample_cols],
#         axis=1,
#         result_type='expand'
#     )

#     gt_df = traw_final.drop(columns=var_cols).set_index('snp_name_full').T.reset_index()
#     gt_df.columns.name = None
#     gt_df.rename(columns={'index':'IID'}, inplace=True)

#     # Process and save frequency info
#     freq = pd.read_csv(label_snps_afreq, sep='\t')
#     freq.rename(columns={'ID':'SNP'}, inplace=True)
#     var_info_df = traw_final.loc[:,var_cols]
#     var_info_df.merge(freq, how='left', on='SNP')
#     var_info_df.to_csv(label_carriers_var_info, index=False)
    
#     # Process and save string format
#     carriers_string = traw_out.drop(columns=var_cols).set_index('snp_name_full').T.reset_index()
#     carriers_string.columns.name = None
#     carriers_string = carriers_string.fillna('./.')
#     carriers_string = carriers_string.astype(str)
#     carriers_string.rename(columns={'index':'IID'}, inplace=True)
#     carriers_string.to_csv(label_carriers_string, index=False)

#     # Process and save integer format
#     carriers_int = traw_final.drop(columns=var_cols).set_index('snp_name_full').T.reset_index()
#     carriers_int.columns.name = None
#     carriers_int.rename(columns={'index':'IID'}, inplace=True)
#     carriers_int.to_csv(label_carriers_int, index=False)




In [6]:
# !gcloud storage cp {data_dir}/outputs/carriers_string_full.csv gs://gp2_carriers/
# !gcloud storage cp {data_dir}/outputs/carriers_int_full.csv gs://gp2_carriers/